##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Recommending movies: retrieval

This tutorial is a slightly adapted version of the [basic retrieval tutorial](https://www.tensorflow.org/recommenders/examples/basic_retrieval#fitting_and_evaluating) from TensorFlow Recommenders documentation.

## Imports


Let's first get our imports out of the way.

In [58]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

ERROR: Could not find a version that satisfies the requirement scann (from versions: none)
ERROR: No matching distribution found for scann


In [89]:
%load_ext tensorboard

In [90]:
!rm -rf ./logs/


In [48]:
import datetime
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

## Preparing the dataset

In [62]:
# Ratings data.
ratings = tfds.load("movielens/100k-ratings", split="train")
# Features of all the available movies.
movies = tfds.load("movielens/100k-movies", split="train")


# Keep only 'movie_title' and 'user_id'
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
})
# Keep only 'movie title'
movies = movies.map(lambda x: x["movie_title"])

Let's use a random split, putting 80% of the ratings in the train set, and 20% in the test set.

In [7]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

Next we figure out unique user ids and movie titles present in the data so that we can create the embedding user and movie embedding tables.

In [75]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

# bring together the batches to find the unique ids and titles
unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

## Implementing a retrieval model

We are going to building a two-tower retrieval model by building each tower separately and then combining them in the final model.

### The query tower

The first step is to decide on the dimensionality of the query and candidate representations:

In [70]:
embedding_dimension = 32


The second is to define the model itself. 

In [71]:
user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

### The candidate tower

We can do the same with the candidate tower.

In [77]:
movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

### Metrics

We use the `tfrs.metrics.FactorizedTopK` metric for our retrieval model.

In [78]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(movie_model)
)

### Loss

The next component is the loss used to train our model. We'll make use of the `Retrieval` task object: a convenience wrapper that bundles together the loss function and metric computation:

In [79]:
# this will default loss to categoricalCrossentropy

task = tfrs.tasks.Retrieval(
  metrics=metrics
)

### The full model

We can now put it all together into a model.

In [80]:
class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.movie_model(features["movie_title"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings)

## Fitting and evaluating

Let's instantiate the model now.

In [82]:
model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.legacy.Adagrad(learning_rate=0.1))

Then shuffle, batch, and cache the training and evaluation data.

In [83]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

Then train the  model:

In [92]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [97]:
%tensorboard --logdir logs/fit

In [98]:
model.fit(cached_train, epochs=20,callbacks=[tensorboard_callback])

Epoch 1/20
10/10 [==============================] - 5s 431ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0063 - factorized_top_k/top_5_categorical_accuracy: 0.0401 - factorized_top_k/top_10_categorical_accuracy: 0.0757 - factorized_top_k/top_50_categorical_accuracy: 0.2697 - factorized_top_k/top_100_categorical_accuracy: 0.4200 - loss: 62823.6026 - regularization_loss: 0.0000e+00 - total_loss: 62823.6026
Epoch 2/20
10/10 [==============================] - 4s 424ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0064 - factorized_top_k/top_5_categorical_accuracy: 0.0402 - factorized_top_k/top_10_categorical_accuracy: 0.0760 - factorized_top_k/top_50_categorical_accuracy: 0.2711 - factorized_top_k/top_100_categorical_accuracy: 0.4223 - loss: 62774.9830 - regularization_loss: 0.0000e+00 - total_loss: 62774.9830
Epoch 3/20
10/10 [==============================] - 5s 467ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0061 - factorized_top_k/top_5_categorical_accurac

Finally, we can evaluate our model on the test set:

In [99]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 2s 162ms/step - factorized_top_k/top_1_categorical_accuracy: 1.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0012 - factorized_top_k/top_10_categorical_accuracy: 0.0044 - factorized_top_k/top_50_categorical_accuracy: 0.0671 - factorized_top_k/top_100_categorical_accuracy: 0.1708 - loss: 32678.5579 - regularization_loss: 0.0000e+00 - total_loss: 32678.5579


{'factorized_top_k/top_1_categorical_accuracy': 0.0001500000071246177,
 'factorized_top_k/top_5_categorical_accuracy': 0.0011500000255182385,
 'factorized_top_k/top_10_categorical_accuracy': 0.004350000061094761,
 'factorized_top_k/top_50_categorical_accuracy': 0.06705000251531601,
 'factorized_top_k/top_100_categorical_accuracy': 0.17075000703334808,
 'loss': 29697.900390625,
 'regularization_loss': 0,
 'total_loss': 29697.900390625}

## Making predictions

Now that we have a model, we would like to be able to make predictions. We can use the `tfrs.layers.factorized_top_k.BruteForce` layer to do this.

In [102]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)

# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)

# Get recommendations.
_, titles = index(tf.constant(["41"]))
print(f"Recommendations for user 41: {titles[0, :5]}")

Recommendations for user 41: [b'Sabrina (1954)' b'Real Genius (1985)' b'Patton (1970)'
 b'Winnie the Pooh and the Blustery Day (1968)'
 b'E.T. the Extra-Terrestrial (1982)']


If you want to leverage ANN techniques to speed up predictions, please check out the [Efficient serving](https://www.tensorflow.org/recommenders/examples/efficient_serving) tutorial.

## Exporting the model

We simply export the `BruteForce` layer we created above:

In [103]:
# Export the query model.
tf.saved_model.save(index, "exported-retrieval/123")

INFO:tensorflow:Assets written to: exported-retrieval/123/assets


INFO:tensorflow:Assets written to: exported-retrieval/123/assets


We will deploy the model with TensorFlow Serving soon.

In [104]:
# Zip the SavedModel folder for easier download
!zip -r exported-retrieval.zip exported-retrieval/ 

  adding: exported-retrieval/ (stored 0%)
  adding: exported-retrieval/123/ (stored 0%)
  adding: exported-retrieval/123/fingerprint.pb (stored 0%)
  adding: exported-retrieval/123/variables/ (stored 0%)
  adding: exported-retrieval/123/variables/variables.data-00000-of-00001 (deflated 12%)
  adding: exported-retrieval/123/variables/variables.index (deflated 34%)
  adding: exported-retrieval/123/saved_model.pb (deflated 81%)
  adding: exported-retrieval/123/assets/ (stored 0%)
